# 1. 算法

## 1.2 算法步骤
所以 K-means 的算法步骤为：

1. 选择初始化的 k 个样本作为初始聚类中心 $a=a_1+a_2+...a_k$ ；
2. 针对数据集中每个样本 $x_i$计算它到 k 个聚类中心的距离并将其分到距离最小的聚类中心所对应的类中；
3. 针对每个类别 $a_j$ ，重新计算它的聚类中心 $a_j=|\frac{1}{c_i}|\sum_{x\in c_i}x$ （即属于该类的所有样本的质心）；
4. 重复上面 2 3 两步操作，直到达到某个中止条件（迭代次数、最小误差变化等）。

## 1.3 复杂度

时间复杂度： $O(tknm)$ ，其中，t 为迭代次数，k 为簇的数目，n 为样本点数，m 为样本点维度。

空间复杂度： $O(m(n+k))$，其中，k 为簇的数目，m 为样本点维度，n 为样本点数。

# 2. 优缺点
## 2.1 优点
容易理解，聚类效果不错，虽然是局部最优， 但往往局部最优就够了；
处理大数据集的时候，该算法可以保证较好的伸缩性；
当簇近似高斯分布的时候，效果非常不错；
算法复杂度低。
## 2.2 缺点
K 值需要人为设定，不同 K 值得到的结果不一样；
对初始的簇中心敏感，不同选取方式会得到不同结果；
对异常值敏感；
样本只能归为一类，不适合多分类任务；
不适合太离散的分类、样本类别不平衡的分类、非凸形状的分类。
# 3. 算法调优与改进
针对 K-means 算法的缺点，我们可以有很多种调优方式：如数据预处理（去除异常点），合理选择 K 值，高维映射等。以下将简单介绍：

## 3.1 数据预处理
K-means 的本质是基于欧式距离的数据划分算法，均值和方差大的维度将对数据的聚类产生决定性影响。所以未做归一化处理和统一单位的数据是无法直接参与运算和比较的。常见的数据预处理方式有：数据归一化，数据标准化。此外，离群点或者噪声数据会对均值产生较大的影响，导致中心偏移，因此我们还需要对数据进行异常点检测。

## 3.2 K值的选择

手肘法：当 K < n 时，曲线急速下降；当 K > n 时，曲线趋于平稳，通过手肘法我们认为拐点 n 为 K 的最佳值。


## 3.3 sklearn 使用
**参数:**
 - n_clusters：int，默认为8;生成的簇数以及生成的质心数
 - init:{"k-means++", "random", ndarray, callable}, 默认为"k-means++";
 - n_init:int,默认为10;k均值算法将在不同质心种子下运行的次数。就惯性而言，最终结果将是n_init个连续运行的最佳输出。
 - max_iter:int,默认为300;单次运行的k均值算法的最大迭代次数。
 
## 3.4 kmeans 示范代码

In [1]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array([[1, 2], [1, 4], [1, 0],
              [10, 2], [10, 4], [10, 0]])
kmeans = KMeans(n_clusters=2, random_state=0, max_iter=200).fit(X)
kmeans.labels_

array([1, 1, 1, 0, 0, 0], dtype=int32)

In [2]:
kmeans.predict([[0, 0], [12, 3]])

array([1, 0], dtype=int32)

In [3]:
kmeans.cluster_centers_

array([[10.,  2.],
       [ 1.,  2.]])

## 3.5 MiniBatchKMeans
对于>= 10k的数据,速度可能更快

这MiniBatchKMeans是该KMeans算法的一种变体，它使用小批量来减少计算时间，同时仍然尝试优化相同的目标函数。小批量是输入数据的子集，在每次训练迭代中随机采样。这些小批量大大减少了收敛到局部解决方案所需的计算量。与减少 k-means 收敛时间的其他算法相比，小批量 k-means 产生的结果通常只比标准算法稍差。

该算法在两个主要步骤之间进行迭代，类似于 vanilla k-means。在第一步中，从数据集中随机抽取样本，形成一个小批量。然后将它们分配给最近的质心。第二步，更新质心。与 k-means 相比，这是在每个样本的基础上完成的。对于小批量中的每个样本，通过取样本和分配给该质心的所有先前样本的流平均值来更新分配的质心。这具有降低质心随时间变化率的效果。执行这些步骤直到收敛或达到预定的迭代次数。

MiniBatchKMeans收敛速度比 快KMeans，但结果的质量降低。在实践中，这种质量差异可能非常小,性比较可以参考：[KMeans 和 MiniBatchKMeans 的比较](https://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html#sphx-glr-auto-examples-cluster-plot-mini-batch-kmeans-py)
